Final Model - Restructured

In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode

In [ ]:
import os

root_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))

data_folder = os.path.join(root_dir, 'csv_files')

data_files = [f for f in os.listdir(data_folder) if f.endswith(('.csv', '.xlsx'))]

dfs = {}
for file in data_files:
    file_path = os.path.join(data_folder, file)
    if file.endswith('.csv'):
        dfs[file] = pd.read_csv(file_path)
    elif file.endswith('.xlsx'):
        dfs[file] = pd.read_excel(file_path, engine='openpyxl')

In [ ]:
df01 = dfs('NHL_Final-01.csv')
df02 = dfs('NHL_Final-02.csv')


In [3]:
df02 = df02.drop(columns = ['GP', 'Pos', 'TK', 'GV', '-9999'])
df02.head()


,Player,CF,CA,CF%,CF% rel,FF,FA,FF%,FF% rel,oiSH%,oiSV%,PDO,oZS%,dZS%,TOI/60,TOI(EV),E+/-,SAtt.,Thru%
0,Wyatt Aamodt,23,37,38.3,-15.0,18,21,46.2,-10.3,10.0,100.0,110.0,20.0,80.0,17:07,16:16,0.1,6,50.0
1,Rodrigo Abols,191,199,49.0,0.3,137,135,50.4,0.3,8.3,84.7,93.0,61.2,38.8,9:11,9:09,-1.4,48,52.1
2,Noel Acciari,729,957,43.2,-8.9,506,658,43.5,-7.6,6.8,88.6,95.4,20.3,79.7,13:46,11:20,-11.9,151,49.7
3,Jack Ahcan,30,32,48.4,-0.9,22,19,53.7,0.1,11.1,93.3,104.4,26.7,73.3,15:48,14:40,0.2,3,33.3
4,Sebastian Aho,1519,968,61.1,2.8,1094,721,60.3,3.3,10.8,85.8,96.7,63.8,36.2,20:06,14:49,9.1,447,46.3


In [4]:
df01['Player'] = df01['Player'].apply(unidecode).str.lower().str.strip()
df02 ['Player'] = df02['Player'].apply(unidecode).str.lower().str.strip()
df03 = pd.merge (df01, df02, how = 'left', on='Player')

In [5]:
df03 = df03.drop(df03[df03['GP'] <= 41].index).reset_index(drop = True)

df03 = df03.replace({'RW': 'F', 'LW': 'F', 'C': 'F'})

df03 = df03.drop (df03[df03['Pos'] == 'G'].index).reset_index(drop = True)

df03 = df03.replace({'--': 0})

df03 = df03.fillna(0)

In [6]:
df03.head()

,Player,Age,Pos,GP,G,A,PTS,+/-,PIM,EVG,...,oiSH%,oiSV%,PDO,oZS%,dZS%,TOI/60,TOI(EV),E+/-,SAtt.,Thru%
0,nikita kucherov,31.0,F,78,37,84,121,22,45,29,...,13.7,90.1,103.9,59.6,40.4,21:11,17:21,16.3,550.0,48.2
1,nathan mackinnon,29.0,F,79,32,84,116,25,41,23,...,11.0,88.9,99.9,63.8,36.2,22:47,18:54,35.1,590.0,54.2
2,leon draisaitl,29.0,F,71,52,54,106,32,34,36,...,10.7,89.9,100.7,60.6,39.4,21:31,18:11,27.3,417.0,57.8
3,david pastrnak,28.0,F,82,43,63,106,0,42,34,...,13.3,87.3,100.5,68.8,31.2,20:11,16:42,1.0,624.0,51.1
4,mitch marner,27.0,F,81,27,75,102,18,14,20,...,13.1,89.9,103.1,48.6,51.4,21:19,16:11,11.6,393.0,44.0


In [7]:
df_final = df03.copy()
df_final.head()
df_final = df_final.drop(columns = ['TOI/60'])

In [8]:
df_final ['TOI(EV)'] = pd.to_timedelta('00:' + df_final['TOI(EV)'].astype(str)).dt.total_seconds()/60

In [9]:
df_final = df_final.fillna(0)
df_final = df_final.replace({'--': 0})
df_final = df_final.drop(columns = ['Age', 'Pos'])

In [10]:
df_final.head()

,Player,GP,G,A,PTS,+/-,PIM,EVG,PPG,SHG,...,FF% rel,oiSH%,oiSV%,PDO,oZS%,dZS%,TOI(EV),E+/-,SAtt.,Thru%
0,nikita kucherov,78,37,84,121,22,45,29,8,0,...,3.8,13.7,90.1,103.9,59.6,40.4,17.350000,16.3,550.0,48.2
1,nathan mackinnon,79,32,84,116,25,41,23,9,0,...,7.8,11.0,88.9,99.9,63.8,36.2,18.900000,35.1,590.0,54.2
2,leon draisaitl,71,52,54,106,32,34,36,16,0,...,7.8,10.7,89.9,100.7,60.6,39.4,18.183333,27.3,417.0,57.8
3,david pastrnak,82,43,63,106,0,42,34,9,0,...,6.2,13.3,87.3,100.5,68.8,31.2,16.700000,1.0,624.0,51.1
4,mitch marner,81,27,75,102,18,14,20,6,1,...,3.0,13.1,89.9,103.1,48.6,51.4,16.183333,11.6,393.0,44.0


In [ ]:
df1 = dfs("NHL_STATS01.xlsx")
df2 = dfs("NHL_STATS02.xlsx")
df3 = dfs("NHL_STATS03.xlsx")
df4 = dfs("NHL_STATS04.xlsx")
df5 = dfs("NHL_STATS05.xlsx")
df6 = dfs("NHL_STATS06.xlsx")
df7 = dfs("NHL_STATS07.xlsx")
df8 = dfs("NHL_STATS08.xlsx")
df9 = dfs("NHL_STATS09.xlsx")
df10 = dfs("NHL_STATS10.xlsx")

df = pd.concat([df1,df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index = True)


In [12]:
df.head()
df_time = df[['Player', 'TOI/GP']]


In [13]:
df_time.head()

,Player,TOI/GP
0,Nikita Kucherov,21:11
1,Nathan MacKinnon,22:47
2,Leon Draisaitl,21:31
3,David Pastrnak,20:11
4,Mitchell Marner,21:19


In [14]:
df_time ['TOI/GP'] = pd.to_timedelta('00:' + df_time['TOI/GP'].astype(str)).dt.total_seconds()/60

C:\Users\huzay\AppData\Local\Temp\ipykernel_528\1213958053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time ['TOI/GP'] = pd.to_timedelta('00:' + df_time['TOI/GP'].astype(str)).dt.total_seconds()/60


In [15]:
df_time['Player'] = df_time['Player'].apply(unidecode).str.lower().str.strip()


C:\Users\huzay\AppData\Local\Temp\ipykernel_528\2280455589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time['Player'] = df_time['Player'].apply(unidecode).str.lower().str.strip()


In [16]:
nhl_df = pd.merge(df_final, df_time, how = 'left', on = 'Player')

In [17]:
nhl_df.loc[nhl_df['Player'] == 'mitch marner', 'TOI/GP'] = 21.3167
nhl_df.loc[nhl_df['Player'] == 'jj peterka', 'TOI/GP'] = 18.1833
nhl_df.loc[nhl_df['Player'] == 'matthew coronato', 'TOI/GP'] = 17.5833
nhl_df.loc[nhl_df['Player'] == 'william cuylle', 'TOI/GP'] = 15.0833
nhl_df.loc[nhl_df['Player'] == 'michael anderson', 'TOI/GP'] = 22.6667
nhl_df.loc[nhl_df['Player'] == 'jacob middleton', 'TOI/GP'] = 21.8667
nhl_df.loc[nhl_df['Player'] == 'pat maroon', 'TOI/GP'] = 11.5333
nhl_df.loc[nhl_df['Player'] == 'emil martinsen lilleberg', 'TOI/GP'] = 15.2833
nhl_df.loc[nhl_df['Player'] == 'nicklaus perbix', 'TOI/GP'] = 14.6833
nhl_df.loc[nhl_df['Player'] == 'cameron york', 'TOI/GP'] = 20.7833
nhl_df.loc[nhl_df['Player'] == 'joe veleno', 'TOI/GP'] = 12.1167
nhl_df.loc[nhl_df['Player'] == 'william borgen', 'TOI/GP'] = 17.0833
nhl_df.loc[nhl_df['Player'] == 'zachary jones', 'TOI/GP'] = 17.25
nhl_df.loc[nhl_df['Player'] == 't.j. brodie', 'TOI/GP'] = 15.6333
nhl_df.loc[nhl_df['Player'] == 'jonathon merrill', 'TOI/GP'] = 14.10
nhl_df.loc[nhl_df['Player'] == 'pierre-olivier joseph', 'TOI/GP'] = 15.2667



In [18]:
nhl_df = nhl_df.drop (columns = 'SAtt.')
nhl_df.head()

,Player,GP,G,A,PTS,+/-,PIM,EVG,PPG,SHG,...,FF% rel,oiSH%,oiSV%,PDO,oZS%,dZS%,TOI(EV),E+/-,Thru%,TOI/GP
0,nikita kucherov,78,37,84,121,22,45,29,8,0,...,3.8,13.7,90.1,103.9,59.6,40.4,17.350000,16.3,48.2,21.183333
1,nathan mackinnon,79,32,84,116,25,41,23,9,0,...,7.8,11.0,88.9,99.9,63.8,36.2,18.900000,35.1,54.2,22.783333
2,leon draisaitl,71,52,54,106,32,34,36,16,0,...,7.8,10.7,89.9,100.7,60.6,39.4,18.183333,27.3,57.8,21.516667
3,david pastrnak,82,43,63,106,0,42,34,9,0,...,6.2,13.3,87.3,100.5,68.8,31.2,16.700000,1.0,51.1,20.183333
4,mitch marner,81,27,75,102,18,14,20,6,1,...,3.0,13.1,89.9,103.1,48.6,51.4,16.183333,11.6,44.0,21.316700


In [19]:
per_60_TOT = ['G', 'A', 'PTS', 'PIM', 'GWG', 'SOG', 'TSA', 'FOW', 'FOL', 'BLK', 'HIT', 'TAKE', 'GIVE']

for item in per_60_TOT:
    nhl_df[item] = (nhl_df[item] / (nhl_df['TOI/GP'] * nhl_df['GP']) * 60).round(5)
    nhl_df = nhl_df.rename(columns = {item: item + '/60'})


per_60_EV = ['EVG', 'EV', 'CF', 'CA', 'FF', 'FA']

for item in per_60_EV:
    nhl_df[item] = (nhl_df[item] / (nhl_df['TOI(EV)'] * nhl_df['GP']) * 60).round(5)
    nhl_df = nhl_df.rename(columns = {item: item + '/60'})

per_game = ['PPG', 'SHG', 'PP', 'SH']

for item in per_game:
    nhl_df[item] = (nhl_df[item] / nhl_df['GP']).round(5)
    nhl_df = nhl_df.rename(columns = {item: item + '/GP'})


In [20]:
nhl_df = nhl_df.drop(columns = ['GP', 'ATOI', 'TOI(EV)', 'TOI/GP'])
nhl_df.head()

,Player,G/60,A/60,PTS/60,+/-,PIM/60,EVG/60,PPG/GP,SHG/GP,GWG/60,...,FA/60,FF%,FF% rel,oiSH%,oiSV%,PDO,oZS%,dZS%,E+/-,Thru%
0,nikita kucherov,1.34358,3.05029,4.39388,22,1.63409,1.28575,0.10256,0.00000,0.32682,...,43.36067,53.0,3.8,13.7,90.1,103.9,59.6,40.4,16.3,48.2
1,nathan mackinnon,1.06674,2.80018,3.86692,25,1.36676,0.92425,0.11392,0.00000,0.16668,...,37.85413,58.7,7.8,11.0,88.9,99.9,63.8,36.2,35.1,54.2
2,leon draisaitl,2.04231,2.12086,4.16317,32,1.33536,1.67310,0.22535,0.00000,0.43203,...,38.29540,59.0,7.8,10.7,89.9,100.7,60.6,39.4,27.3,57.8
3,david pastrnak,1.55888,2.28394,3.84282,0,1.52263,1.48970,0.10976,0.00000,0.14501,...,41.62407,51.7,6.2,13.3,87.3,100.5,68.8,31.2,1.0,51.1
4,mitch marner,0.93823,2.60620,3.54443,18,0.48649,0.91544,0.07407,0.01235,0.24325,...,44.12404,50.2,3.0,13.1,89.9,103.1,48.6,51.4,11.6,44.0


In [21]:
nhl_df01 = nhl_df.copy()

names_nhl = nhl_df01['Player']

nhl_df01 = nhl_df01.drop (columns = ['Player'])

In [22]:
salaries_df = pd.read_csv('salaries.csv')

salaries_df = salaries_df.drop(columns = ['0', '1', '2'])

salaries_df['Player'] = salaries_df['Player'].apply(unidecode).str.lower().str.strip()

names_df = pd.DataFrame({'Player' : names_nhl})

In [23]:
names_df = pd.merge(names_df, salaries_df, how = 'left', on = 'Player')

In [24]:
names_df.loc[names_df['Player'] == 'mitch marner', 'Salary'] = 10.903
names_df.loc[names_df['Player'] == 'jj peterka', 'Salary'] = 0.855834
names_df.loc[names_df['Player'] == 'matthew coronato', 'Salary'] = 0.925 
names_df.loc[names_df['Player'] == 'william cuylle', 'Salary'] = 0.828333
names_df.loc[names_df['Player'] == 'zachary bolduc', 'Salary'] = 0.863334
names_df.loc[names_df['Player'] == 'michael anderson', 'Salary'] = 4.125
names_df.loc[names_df['Player'] == 'emil martinsen lilleberg', 'Salary'] = 0.87
names_df.loc[names_df['Player'] == 'chris tanev', 'Salary'] = 4.5
names_df.loc[names_df['Player'] == 'alexey toropchenko', 'Salary'] = 1.25
names_df.loc[names_df['Player'] == 'fedor svechkov', 'Salary'] = 0.925
names_df.loc[names_df['Player'] == 'joe veleno', 'Salary'] = 0.9
names_df.loc[names_df['Player'] == 'egor zamula', 'Salary'] = 1.7
names_df.loc[names_df['Player'] == 'j.j. moser', 'Salary'] = 3.375
names_df.loc[names_df['Player'] == 'artem zub', 'Salary'] = 4.6
names_df.loc[names_df['Player'] == 't.j. brodie', 'Salary'] = 0.775
names_df.loc[names_df['Player'] == 'mathew dumba', 'Salary'] = 3.75
names_df.loc[names_df['Player'] == 'marc del gaizo', 'Salary'] = 0.775
names_df.loc[names_df['Player'] == 'emil andrae', 'Salary'] = 0.903
names_df.loc[names_df['Player'] == 'jonathon merrill', 'Salary'] = 1.2
names_df.loc[names_df['Player'] == 'devin shore', 'Salary'] = 0.775
names_df.loc[names_df['Player'] == 'zack ostapchuk', 'Salary'] = 0.825
names_df.loc[names_df['Player'] == 'matty beniers', 'Salary'] = 7.142857
names_df.loc[names_df['Player'] == 'oliver wahlstrom', 'Salary'] = 1.0



In [25]:
names_df.head()

,Player,Salary
0,nikita kucherov,9.500
1,nathan mackinnon,12.600
2,leon draisaitl,8.500
3,david pastrnak,11.250
4,mitch marner,10.903


In [26]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

nhl_scaled = scale.fit_transform(nhl_df01)

In [27]:
from sklearn.neighbors import NearestNeighbors

sim_players = NearestNeighbors(n_neighbors = 550).fit(nhl_scaled)

In [28]:
sample = 'charlie coyle'
index_pos = names_df.index.get_loc(names_df[names_df['Player'] == sample].index[0])
print (index_pos)

241


In [29]:
result_comp = sim_players.kneighbors([nhl_scaled[241]])

In [30]:
print(names_df['Salary'].loc[185])



6.0


In [31]:
print(result_comp)

(array([[ 0.        ,  4.35769553,  4.72996267,  5.08349219,  5.12981758,
         5.23865099,  5.35790137,  5.44198722,  5.53355458,  5.53690557,
         5.57862974,  5.57917068,  5.60925424,  5.66961908,  5.6780027 ,
         5.68799195,  5.69123854,  5.78635639,  5.81179635,  5.81740418,
         5.88970887,  5.91386535,  5.94245679,  5.99112876,  5.99388346,
         5.99523703,  6.00367612,  6.04233449,  6.08740897,  6.12707514,
         6.14282762,  6.19079225,  6.20717004,  6.24603051,  6.28012167,
         6.29480934,  6.31100582,  6.3228486 ,  6.36261668,  6.3704732 ,
         6.37397932,  6.39478351,  6.41028302,  6.41658562,  6.42291996,
         6.45230523,  6.46539565,  6.4893434 ,  6.51072621,  6.51701583,
         6.53274228,  6.53426141,  6.53891097,  6.54446822,  6.54665771,
         6.55544572,  6.56813724,  6.58376954,  6.59772639,  6.63078607,
         6.65154733,  6.68312026,  6.69836646,  6.73425777,  6.74348381,
         6.75021967,  6.75602411,  6.77634632,  6.

In [32]:
print (result_comp [1][0][1])

185


In [33]:
def comparables(comp_name, salary_max):

    #retrives position of comparison player in original array
    position = names_df.index.get_loc(names_df[names_df['Player'] == comp_name].index[0])

    #creates an array to score the most comparable players to the requested comparison
    comp_array = sim_players.kneighbors([nhl_scaled[position]])

    #variable assignment
    index = 1
    player_count = 0
    final_dict = {
        "name": [],
        "salary": [], 
        "similar_score": []
        }
    

    #for loop 
    for i in range(595): 

        #stores position of comparable
        pos_comp = comp_array[1][0][index]

        if names_df['Salary'].loc[pos_comp] <= salary_max:

            final_dict["name"].append(names_df['Player'].loc[pos_comp])
            final_dict["salary"].append(names_df['Salary'].loc[pos_comp].round(4))
            final_dict["similar_score"].append(comp_array[0][0][index].round(4))

            player_count += 1
            
        index += 1

        if player_count == 3:
            return final_dict
    
    


In [34]:
print (comparables ("brady tkachuk", 8))

{'name': ['sam bennett', 'pavel dorofeyev', 'kevin fiala'], 'salary': [np.float64(4.425), np.float64(1.835), np.float64(7.875)], 'similar_score': [np.float64(5.3201), np.float64(5.4997), np.float64(5.8061)]}
